In [122]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import scipy as spy
from scipy import linalg
import pylab as pylab

In [3]:
r"""
This module generates trajectories of a simple two dimensional toy model for testing purposes.
"""

import numpy as np

__all__ = ['generate_test_data']

def _gradient(x, y):
    return (x * x - 1.0) * 4.0 * x + 0.5, (4.0 * y * y - 7.0) * y

def _bd(x0, y0, length, dt=0.005):
    coeff_A = dt
    coeff_B = np.sqrt(2.0 * dt)
    x = [x0]
    y = [y0]
    for _i in range(1, length):
        dx, dy = _gradient(x[-1], y[-1])
        x.append(x[-1] - coeff_A * dx + coeff_B * np.random.normal())
        y.append(y[-1] - coeff_A * dy + coeff_B * np.random.normal())
    return np.array([[_x, _y] for _x, _y in zip(x, y)], dtype=np.float64)

def generate_test_data(traj_length=20000, num_trajs=5):
    r"""
    This functions handles the test data generation via Brownian dynamics simulations with
    randomized starting configurations.
    Parameters
    ----------
    traj_length : int, optional, default=20000
        Length of a single trajectory.
    num_trajs : int, optional, default=5
        Number of independent trajectories.
    Returns
    -------
    trajs : list of numpy.ndarray(shape=(traj_length, 2), dtype=numpy.float64) objects
        Time series of configurations of the toy model.
    """
    trajs = []
    for _i in range(num_trajs):
        trajs.append(_bd(3.0 * np.random.rand() - 1.5, 3.0 * np.random.rand() - 1.5, traj_length))
    return trajs

In [46]:
"""
TIMESCALES
"""

def stat(T):
    w, v = np.linalg.eig(T.T)
    
    j_stat =np.argmin(abs(w-1.0))
    p_stat=v[:,j_stat].real
    p_stat /= p_stat.sum()
    return p_stat

In [5]:
"""
TICA
"""

def make_data_mean_free(X):
    """
    
    """
    
    n_col = len(X[0])

    for j in range(n_col):
        med=((X[:,j]).sum())/(len(X[:,j]))
        X[:,j] -= med
    return X

def TICA(X,lag):
    """
    Time-lagged/ time-structure-based independent component analysis.
    
    INPUT:
        - X = As input, we consider a d-dimensional vector, called r(t)=(ri(t))i=1,...,D. 
        Here, t is an integer from {1...N} denoting the time step. That is, we have a N x D matrix.
        
    OUTPUT:
        - 
    """
    
    X_ = np.copy(X)
    X_ = make_data_mean_free(X)
    
    T=len(X)
    D=len(X[0])
    
    X_0 = X_[[i for i in range(T-lag)],:]
    X_lag = X_[[lag+i for i in range(T-lag)],:]
    
    #Covariance matrices
    C_0 = (1/(T-lag-1))*(np.dot(np.transpose(X_0),X_0))
    C_lag = (1/(T-lag-1))*(np.dot(np.transpose(X_0),X_lag))
    
    #Symmetrized time-lagged covariance matrix
    C_lag = 1/2 * (C_lag + np.transpose(C_lag))
    
    #Eigenvalue problem --> C_lag * U = eig_val * C_0 * U]
    U = spy.linalg.eig(C_lag,C_0)[1]
    
    #Projection onto the TICA space
    Z = np.dot(X_,U)
    
    return Z

In [6]:
"""
K-MEANS
"""

import os
import numpy as np

# kmeans clustering algorithm
# data = set of data points
# k = number of clusters
# c = initial list of centroids (if provided)
#
def kmeans(data, k, c):
    centroids = []

    centroids = randomize_centroids(data, centroids, k)  

    old_centroids = [[] for i in range(k)] 

    iterations = 0
    while not (has_converged(centroids, old_centroids, iterations)):
        iterations += 1

        clusters = [[] for i in range(k)]

        # assign data points to clusters
        clusters = euclidean_dist(data, centroids, clusters)

        # recalculate centroids
        index = 0
        for cluster in clusters:
            old_centroids[index] = centroids[index]
            centroids[index] = np.mean(cluster, axis=0).tolist()
            index += 1
    print("The total number of data instances is: " + str(len(data)))
    print("The total number of iterations necessary is: " + str(iterations))
    print("The means of each cluster are: " + str(centroids))
    print("The clusters are as follows:")
    for cluster in clusters:
        print("Cluster with a size of " + str(len(cluster)) + " starts here:")
        print(np.array(cluster).tolist())
        print("Cluster ends here.")

    return

# Calculates euclidean distance between
# a data point and all the available cluster
# centroids.      
def euclidean_dist(data, centroids, clusters):
    for instance in data:  
        # Find which centroid is the closest
        # to the given data point.
        mu_index = min([(i[0], np.linalg.norm(instance-centroids[i[0]])) \
                            for i in enumerate(centroids)], key=lambda t:t[1])[0]
        try:
            clusters[mu_index].append(instance)
        except KeyError:
            clusters[mu_index] = [instance]

    # If any cluster is empty then assign one point
    # from data set randomly so as to not have empty
    # clusters and 0 means.        
    for cluster in clusters:
        if not cluster:
            cluster.append(data[np.random.randint(0, len(data), size=1)].flatten().tolist())

    return clusters


# randomize initial centroids
def randomize_centroids(data, centroids, k):
    for cluster in range(0, k):
        centroids.append(data[np.random.randint(0, len(data), size=1)].flatten().tolist())
    return centroids


# check if clusters have converged    
def has_converged(centroids, old_centroids, iterations):
    MAX_ITERATIONS = 1000
    if iterations > MAX_ITERATIONS:
        return True
    return old_centroids == centroids

def kmeans2(data, k, c):
    centroids = []

    centroids = randomize_centroids(data, centroids, k)  

    old_centroids = [[] for i in range(k)] 

    iterations = 0
    while not (has_converged(centroids, old_centroids, iterations)):
        iterations += 1

        clusters = [[] for i in range(k)]

        # assign data points to clusters
        clusters = euclidean_dist(data, centroids, clusters)

        # recalculate centroids
        index = 0
        for cluster in clusters:
            old_centroids[index] = centroids[index]
            centroids[index] = np.mean(cluster, axis=0).tolist()
            index += 1
    return centroids,clusters

def cloust_list(L, clus_0, a_0, w):
    for i in range(len (clus_0)): 
        for k in range(len (a_0)):
            if  all(a_0[k]==clus_0[i]):
                L[k]=w
    return L

def Clustering(traj,nclus):
    cen, clus = kmeans2(traj, nclus, 5)
    L=[0 for i in range(len(traj))]
    for  b in range(nclus):
        cloust_list(L, clus[b],traj,b)
    return L

In [7]:
"""
COUNTMATRIX
"""

def sliding_window(traj,lag=1):
    """
    Get the count matrix out of the trajectory from clusters.
    
    INPUT:
        traj = A list. A sucesion of states.
        lag = Lagtime we are considering.
        
    OUTPUT:
        A count matrix.
    """
    
    dim = max(traj)+1
    C=np.zeros([dim,dim])
    for i in range(len(traj)-lag):
        C[traj[i],traj[i+lag]] +=1
        
    return C

In [195]:
"""
KOSARAJU
"""

def stat(T):
    w, v = np.linalg.eig(T.T)
    
    j_stat =np.argmin(abs(w-1.0))
    p_stat=v[:,j_stat].real
    p_stat /= p_stat.sum()
    return p_stat

def obtain_active_set(T):
    """
    Function for other parts of the project. It gets the biggest connected component of the matrix 
    that we are given.
    
    INPUT:
        - T = The probability transition matrix of the markov model.
    
    OUTPUT:
        - C = A square matrix. The biggest connected component of the matrix.
        - L = A list of vertices. The states of T that correspond to the biggest component.
    
    """
    
    b=0
    j=0
    components=kosarajus_algo2(T)
    for i in components:
        a=len(components[i])
        if a>b:
            b=a
            j=i
    L=list(components[j])
    L=np.sort(L)
    C=np.array([T[i,:] for i in L])
    C=np.array([C[:,i] for i in L])
    C=np.transpose(C)
    return (C,L)

def Assign2(u,root,LIST,components,M):
    """
    Recursive subfunction for kosarajus
    Strong components are to be represented by appointing a separate root vertex for each component,
    and assigning to each vertex the root vertex of its component.
    
    INPUT:
        
        - u = An integer, which represents a vertex (in our numeration) that has to be
        assigned to some component.
        - root = An integer, which represents a component.
        
        - LIST = A list of vertices that are not yet introduced in the dictionary.
        
        - components = A dictionary containing the vertices (numerated from 0 to n-1), 
        each vertex associated to the root representing its component.
        - M = A transition matrix (which is the adjacency matrix of a graph).
    
    OUTPUT:
    
        - It just changes the dictionary components, assigning to each vertex its root.
    
    """
    
    in_=[i for i in M[:,u]]
    
    if u in LIST:
        
        if not root in components:
            components[root]=[u]
        elif root in components:
            components[root].append(u)
        LIST.remove(u)
            
        for i in range(len(in_)):
            if not(in_[i]==0):
                Assign2(i,root,LIST,components,M)
    return

def Visit(u,Visited,L,M):
    """
    Recursive subfunction for kosarajus
    
    INPUT:
        
        - u = An integer, which represents a vertex (in our numeration).
        - Visited = A list of the vertices already visited.
        - L = an ordered list of graph vertices, that will grow to contain each vertex once.
        - M = A transition matrix (which is the adjacency matrix of a graph).
    
    OUTPUT:
        
        - It just adds in order vertices to the list L.
    
    """
    out=M[u,:]
    if not(u in Visited):
        Visited.append(u)
        for i in range(len(out)):
            if not(out[i]==0):
                Visit(i,Visited,L,M)
        L.insert(0,u) 
    return

def kosarajus_algo2(M):
    """
    Returns a dictionary containing the vertices and their inclusion in strong components.
    Strong components are to be represented by appointing a separate root vertex for each component,
    and assigning to each root the list of vertices inside that component.
    If the graph is represented as an adjacency matrix, the algorithm requires Ο(V^2) time.
    
    INPUT:
    
        - M = A transition matrix (which is the adjacency matrix of a graph).
    
    OUTPUT:
    
        - components = A dictionary containing the components (numerated from 0 to ..), 
        each root associated to a list of vertices that are part of that component.
    
    """
    
    Visited=[]
    L=[]
    
    components={}
    
    Vertices= [i for i in range(len(M[:,1]))]
    LIST=list(Vertices)
    
    for i in Vertices:
        Visit(i,Visited,L,M)
    for u in L:
        Assign2(u,u,LIST,components,M)
    return components  

In [185]:
"""
REVERSIBLE ESTIMATOR
"""

def normalize(M):
    """
    Subfunction for T. It normalizes the matrix given as input.
    
    INPUT:
        - M = A matrix M.
        
    OUTPUT:
        - M0 = The matrix M normalized, with rows that add to 1.     
    """
    
    M0=np.array(M)
    if M0.ndim == 1:
        s= M0.sum()
        return np.divide(M0,s)
        
    elif M0.ndim == 2:
        s=M0.sum(axis=1)
        return np.divide(M0,s[:,np.newaxis])
    else:
        return "Normalize. Wrong input"

def T_non_reversible(C):
    """
    Function to get the transition matrix from the count matrix. It simply normalizes the count matrix.
    Is easy, and useful for very large amount of data.
    
    INPUT:
        - C = Count matrix.
    
    OUTPUT:
        - P = The probability transition matrix of the markov model.
    """
    
    return normalize(C)

def T_reversible(C,max_iterations=100,error=0.1):
    """
    Function to get the transition matrix from the count matrix. It gives a matrix that is reversible.
    That is, the markov model obtained is reversible (it satisfies the detailed balance equations).
    Detailed balance implies that, around any closed cycle of states, there is no net flow of probability. 
    For example, it implies that, for all a, b and c,
    T( a , b ) T( b , c ) T( c , a ) = T( a , c ) T( c , b ) T( b , a ).
    
    INPUT:
        - C = Count matrix constructed with lag tau.
        - max_iterations = maximum number of iterations we allow.
        - error = error that we consider to understand that the iteration has converged.
        
    OUTPUT:
        - P = The probability transition matrix of the markov model.
    """
    
    C_i = C.sum(axis=1) #array of the sums of the rows of C
    C_j = C.sum(axis=0) #array of the sums of the columns of C
    
    P = T_non_reversible(C)
    P = (obtain_active_set(P))[0]
    pi = stat(P)
    
    P=np.multiply(pi,P)
    X_0= P #initial state
    
    it=0
    Er=0.2 #TO BE CHANGED
    
    while (Er > error) and (it< max_iterations):
        Xi_0= X_0.sum(axis=1) #array of the sums of the rows of X_0
        Xj_0= X_0.sum(axis=0) #array of the sums of the rows of X_0
        
        
        X_1= C + np.matrix.transpose(C)
        if not(len(C_i)==len(Xi_0)):
            return X_0
        I = np.divide((C_i),(Xi_0))
        if not(len(C_j)==len(Xj_0)):
            return X_0
        II = np.divide((C_j),(Xj_0))
        denominador = I + II
        X_1=np.divide(X_1,denominador)
        
        X_0 = X_1
        it+=1
        
    P = normalize(X_1)
    return P

In [224]:
trajs = generate_test_data(2000, 5)
traj0 = trajs[0]
traj0[[i for i in range(20)],:]

array([[ 0.99645136, -0.57337447],
       [ 0.99649725, -0.54816479],
       [ 0.7907901 , -0.39085369],
       [ 0.84911771, -0.33618201],
       [ 0.80381854, -0.43284228],
       [ 0.78674415, -0.39080482],
       [ 0.69342014, -0.29499768],
       [ 0.78248291, -0.22753419],
       [ 0.88125948, -0.11262015],
       [ 0.91095357, -0.07005534],
       [ 0.85901036, -0.07614186],
       [ 0.98301002, -0.18656752],
       [ 1.0877931 , -0.32901274],
       [ 1.08109104, -0.42256256],
       [ 0.92115128, -0.59793316],
       [ 0.84354598, -0.62827661],
       [ 0.84569415, -0.50790612],
       [ 0.91894507, -0.47417158],
       [ 0.7730035 , -0.50618696],
       [ 0.90751057, -0.2807605 ]])

In [222]:
def plot_timescales(T,LAG,n_timescales):
    """
    Functio
    
    INPUT:
        - T = A list of markov transition matrices such that T[i] = T(lag), transition matrix at lag time lag.
        - LAG = A list of the lagtimes in which we plot the timescales
    """
    
    if len(T)==len(LAG):
        k=len(LAG)
    else:
        return "Input dimensions dont match"
    
    kk=len(implied_timescales(T[0],LAG[0])) 
    timescale_trajs=[]
    for i in range(kk):
        timescale_trajs.append([])
        
    for l in range(k):
        t_lag=implied_timescales(T[l],LAG[l]) #timescales at lag l
        
        for j in range(len(t_lag)):
            if t_lag[j]<1000:
                (timescale_trajs[j]).append(t_lag[j])
            else:
                (timescale_trajs[j]).append(timescale_trajs[j][-1])
    
    #plot slower n_timescales
    if n_timescales < len(timescale_trajs):
        n = n_timescales
    else:
        n = len(timescale_trajs)
    for i in range(n):
        if len(LAG) == len(timescale_trajs[i]):
            py.plot(LAG,timescale_trajs[len(timescale_trajs)-i-3],'o',linestyle='-')
        else:
            next
    py.xlabel('Lag time')
    py.ylabel('Implied timescale')
    py.show() 

In [201]:
def implied_timescales(T_lag,lag):
    """
    Calculates timescale_i(k*lag)=
    
    INPUT:
        - T = A markov transition matrix at a lagtime lag.
        - lag = reference lagtime we are considering.
    """
    
    eig_val, eig_vec=np.linalg.eig(T_lag)
    eig_val = np.absolute(eig_val)
    eig_val = np.sort(eig_val)
    if eig_val[0] == 0:
        eig_val[0] = eig_val[1]
    L=[]
    for i in range(len(eig_val)):
        if (eig_val[i]==0):
            eig_val[i] = 0.01
        ti = -(lag)/(np.log(eig_val[i]))
        if len(L) == 0:
            L.append(ti)
        else:
            if ti < 0:
                L.append(L[i-1])
            elif ti > 10 * L[i-1]:
                L.append(L[i-1])
            else:
                L.append(ti)
    return(L)

In [225]:
LAG=[1,2,3,5,10,15,20,30,40,50,70,90,110,130,150,175,200]
#LAG=[1,2,3,5,10,15,20,30,40,50]
n_centers = 50
Ts = []
L = Clustering(traj0,n_centers)

for i in range(len(LAG)):
    C_lag = sliding_window(L,LAG[i])
    T_lag = T_reversible(C_lag)
    Ts.append(T_lag)
plot_timescales(Ts,LAG,10)

In [149]:
C_lag = sliding_window(L,35)
T_lag = T_reversible(C_lag)
T=T_lag
T

array([[ 0.02631579,  0.        ,  0.01315789,  0.22368421,  0.30263158,
         0.        ,  0.07894737,  0.02631579,  0.        ,  0.32894737],
       [ 0.        ,  0.15625   ,  0.        ,  0.140625  ,  0.015625  ,
         0.1875    ,  0.        ,  0.15625   ,  0.34375   ,  0.        ],
       [ 0.02272727,  0.        ,  0.        ,  0.38636364,  0.11363636,
         0.        ,  0.38636364,  0.06818182,  0.        ,  0.02272727],
       [ 0.14285714,  0.07563025,  0.14285714,  0.01680672,  0.04201681,
         0.13445378,  0.14285714,  0.16806723,  0.07563025,  0.05882353],
       [ 0.27058824,  0.01176471,  0.05882353,  0.05882353,  0.07058824,
         0.05882353,  0.2       ,  0.08235294,  0.05882353,  0.12941176],
       [ 0.        ,  0.12244898,  0.        ,  0.16326531,  0.05102041,
         0.16326531,  0.05102041,  0.18367347,  0.25510204,  0.01020408],
       [ 0.0483871 ,  0.        ,  0.13709677,  0.13709677,  0.13709677,
         0.04032258,  0.16129032,  0.13709677

In [150]:
kosarajus_algo2(T)


{0: [0, 2, 3, 1, 4, 5, 6, 7, 8, 9]}

In [130]:
def forward_commitors(T,A,B):
    """
    Formally defined as the probability to hit set B rather than A
    
    INPUT:
        - T = A markov transition matrix
        - A = A subset of states.
        - B = A subset of states.
        
    OUTPUT:
    """

        
    n = len(T)
    
    q=np.zeros(n)
    q[B] = np.ones(len(B))
    
    B_ = [i for i in range(n) if i not in B]
    AB_ = [i for i in B_ if i not in A]
    
    
    a = np.copy(T)
    a = a[:,AB_]
    a = a[AB_,:]
    
    I=np.identity(len(a))
    a=a-I
    
    b = np.copy(T)
    b = b[:,B]
    b = b[AB_,:]

    b=b.sum(axis=1)
    b= -1*b 
    
    if not(len(a)==0 or len(b)==0):
        Q = (np.linalg.lstsq(a,b))[0]
        h[AB_] = Q
    
    return q        

def probability_current(T,A,B):
    """
    ¡¡NOT OPTIMIZED!!
    
    The probability current gives the averege number of reactive trajectories
    (those going from A to B, without entering A before reaching B) that transition
    from i to j per time unit.
    
    """
    
    n = len(T)
    
    F = np.copy(T)
    
    q = forward_commitors(T,A,B)   
    q_ = forward_commitors(T,B,A)
    
    pi=stat(T)
    
    for i in range(n):
        for j in range(n):
            if j==i:
                F[i,j]=0
            else:
                F[i,j] = (pi[i])*(q_[i])*(T[i,j])*(q[j])
    
    return F

def average_current(T,A,B):
    """
    ¡¡NOT OPTIMIZED!!
    
    Average total number of trajectories going from A to B per time unit.
    
    INPUT:
        - T = A markov transition matrix.
        - A = A subset of indices.
        - B = A subset of indices.
    
    """
    
    f_AB = probability_current(T,A,B)
        
    n=len(f_AB)
    F_AB = np.zeros([n,n])
    
    for i in A:
        for j in B:
            F_AB[i,j] = f_AB[i,j]
    F_AB=F_AB.sum(axis=1)
    F_AB=F_AB.sum(axis=0)
    
    return F_AB

def transition_rates(T,A,B):
    """
    ¡¡NOT OPTIMIZED!!
    
    The forward and backward commitor are sufficient to calculate transition rates between 
    the sets A and B
    
    """
    
    F_AB = average_current(T,A,B)
    pi = stat(T)
    q = forward_commitors(T,A,B)
    
    sum=0
    
    for j in B:
        sum= sum + (pi[j])*(q[j])
        
    return (F_AB/sum)

def mean_first_passage_times(T,A,B):
    """
    
    """
    
    t= transition_rates(T,A,B)
    return 1/t

In [137]:
def hitting_time_ij(T,i,j,steps):
    """
    Calculates the hitting time between states i, j from the markov transition matrix.
    Since p_ij is the probability of going to state j given that we are in state 
    i, the product π_i * p_ij is the long run proportion of transitions that go from state
    i to state j.
    
    INPUT:
        - T = A markov transiton matrix.
        - i = Some state from which we start.
        - j = Some state where we wanna arrive.
        - steps = An integer. Max number of steps we want to consider.
        
    OUTPUT:
        - A picture showing how hitting probability between i and j changes with time.
        - Limiting probability of going from state i to state j.

    """
    
    A=T.copy()
    
    hit_idx = (i,j)

    # Make the final state an absorbing condition
    A[hit_idx[1],:] = 0
    A[hit_idx[1],hit_idx[1]] = 1

    # Make a proper Markov matrix by row normalizing
    A = (A.T/A.sum(axis=1)).T
    
    B = A.copy()
    Z = []
    for n in range(steps):
        Z.append( B[hit_idx] )
        B = py.dot(B,A)
    
    py.plot(Z)
    py.xlabel("steps")
    py.ylabel("hit probability")
    py.show()

In [134]:
A=[0,1,2]
B=[i for i in range(len(T)) if not(i in A)]
mean_first_passage_times(T,A,B)

3.6715251164100229

In [151]:
eig_val, eig_vec = np.linalg.eig(T)
eig_val

array([ 1.        ,  0.58854462, -0.41339241,  0.24613131, -0.29267023,
       -0.14806128, -0.10302024, -0.05160537,  0.03622155,  0.01540873])